In [ ]:
import pandas as pd

df_k = pd.read_parquet('data/pt_k.parquet')
df_li = pd.read_parquet('data/pt_li.parquet')

df_li

In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

def process(edl_constructor, dataframe: pd.DataFrame, pzc_she: float):
    """
    Process Monteiro data and calculate phi0-phi'
    """
    dataframe["E_SHE"] = dataframe["E_RHE"] - dataframe["pH"] * 59e-3
    dataframe["phi0"] = dataframe["E_SHE"] - pzc_she

    # Find unique log concentration values
    unique_logc_values = dataframe.drop_duplicates(subset="log c")["log c"].values

    # One computation for each concentration value
    processed_dfs = []

    for logc in unique_logc_values:
        df_select_logc = dataframe[dataframe["log c"] == logc].copy()
        model = edl_constructor(10**logc)

        potentials_vs_pzc = np.linspace(np.min(df_select_logc["phi0"]), 0, 50)
        sweep = model.potential_sweep(potentials_vs_pzc, tol=1e-4)

        # Interpolate the result for the phi0 values already in the dataframe
        interpolator = interp1d(potentials_vs_pzc, sweep["phi_rp"])
        df_select_logc["phi_rp"] = interpolator(df_select_logc["phi0"])
        processed_dfs.append(df_select_logc)

    return_df = pd.concat(processed_dfs)
    return_df['c'] = 10 ** return_df['log c'].values
    return return_df

In [ ]:
from edl.models import DoubleLayerModel as model2
from edl.constants import PT_PZC_SHE_V as PZC

species_dict = {'Li': 7, 'K': 5}
df_dict = {}

for species, gamma in species_dict.items():
    df = pd.read_parquet(f"data/pt_{species}.parquet")
    df = process(lambda c: model2(c, gamma, 2), df, pzc_she=PZC)
    df_dict[species] = df

In [ ]:
for species, df in df_dict.items():
    df['species'] = species
all_df = pd.concat(list(df_dict.values()))

In [ ]:
all_df.to_csv('data/pt_df.csv', index=False, float_format='%10.3f')